# Stochastic Simulation: Discrete Event Simulation

In [1]:
import math
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as st
import random as rd
import statsmodels.api as sm
from typing import Union
from scipy.optimize import fsolve

import TidySimStat as tss

# from importlib import reload  
# tss = reload(tss)

## 1. Output Detailed Simulation Results

In [6]:
ser = tss.Servers(f_serve=lambda: tss.sim_exp(8), f_inter=lambda: tss.sim_exp(1), num_servers=10, cap_queue=0)

In [25]:
ser.advance()

6.753795326739636

In [26]:
ser.collect()

[7.720396663184743,
 8.840138224936409,
 9.484530404618432,
 10.860439852394792,
 11.331508692755676,
 15.317543844001383,
 16.496832281135767]

In [24]:
ser.pd_events

,clock,type,len_queue,num_arriveds,num_busys,num_block,i_customer
1,2.094439,arrive-serve,0,1,1,0,1
2,2.096849,arrive-serve,0,2,2,0,2
3,2.105684,arrive-serve,0,3,3,0,3
4,2.911973,arrive-serve,0,4,4,0,4
5,3.292661,arrive-serve,0,5,5,0,5
6,5.549782,arrive-serve,0,6,6,0,6
7,6.075445,arrive-serve,0,7,7,0,7


## 2. Calculate Block Rates of Queueing System with Finite Waiting Space

In [2]:
num_runs = 100
block_rates = [0 for i in range(num_runs)]
block_rates_warmup = [0 for i in range(num_runs)]
for i in range(num_runs):
    ser = tss.Servers(f_serve=lambda: tss.sim_exp(8), f_inter=lambda: tss.sim_exp(1), num_servers=10, cap_queue=0)
    while ser.num_arriveds <= 100:
        ser.advance()
    block_rates_warmup[i] = ser.block_rate_warmup
    block_rates[i] = ser.block_rate

In [3]:
# tss.cal_mean_sample(block_rates)
tss.est_interval(block_rates)

[0.09599774813880696, 0.11608145978198514]

In [ ]:
num_runs = 10
block_rates = [0 for i in range(num_runs)]
for i in range(num_runs):
    ser = tss.Servers(f_serve=lambda: tss.sim_exp(8), f_inter=lambda: st.erlang(1), num_servers=10, cap_queue=0)
    while ser.num_arriveds <= 10000:
        ser.advance()
    block_rates[i] = ser.block_rate

## 3. Theoretical Results

In [4]:
def cal_erlangs_b(num_servers, a):
    prob_block = a**num_servers / math.factorial(num_servers) / \
        sum([a**i / math.factorial(i) for i in range(0, num_servers+1)])
    return prob_block

In [5]:
cal_erlangs_b(10, 8)

0.12166106425295149

In [16]:
st.erlang(loc=1)

TypeError: _parse_args() missing 1 required positional argument: 'a'

In [15]:
help(st.erlang)

Help on erlang_gen in module scipy.stats._continuous_distns object:

class erlang_gen(gamma_gen)
 |  erlang_gen(momtype=1, a=None, b=None, xtol=1e-14, badvalue=None, name=None, longname=None, shapes=None, extradoc=None, seed=None)
 |  
 |  An Erlang continuous random variable.
 |  
 |  %(before_notes)s
 |  
 |  See Also
 |  --------
 |  gamma
 |  
 |  Notes
 |  -----
 |  The Erlang distribution is a special case of the Gamma distribution, with
 |  the shape parameter `a` an integer.  Note that this restriction is not
 |  enforced by `erlang`. It will, however, generate a warning the first time
 |  a non-integer value is used for the shape parameter.
 |  
 |  Refer to `gamma` for examples.
 |  
 |  Method resolution order:
 |      erlang_gen
 |      gamma_gen
 |      scipy.stats._distn_infrastructure.rv_continuous
 |      scipy.stats._distn_infrastructure.rv_generic
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  fit(self, data, *args, **kwds)
 |      Return MLEs for sha

In [30]:
li = [1, 2, -1]
li = [i > 0 for i in li]
sum(li)

2

In [57]:
def sim_exp_hyper(pmf:list, expects:list, n:int, whi_seed:int=123) -> list:
    """Simulate realisation of hyper-exponentially distributed random
    variables using inverse transformation method.

    Keyword Arguments
    =================
    pmf: probability mass function
    expects: expectation of each individual exponential distribution
    n: number of realisations
    whi_seed: seed
    """

    if len(pmf) != len(expects):
        raise ValueError("len(pmf) != len(expects).")
#     elif not equal(sum(pmf), 1):
#         raise ValueError("The sum of probablity mass function is not 1.")
    elif sum([i < 0 for i in pmf]) > 0:
        raise ValueError(f"There are negative values in the probablity mass "
            f"function {pmf}.")

    def get_eq(u, x):
        eq = 1 - sum(pmf[i] * math.exp(- x / expects[i]) for i in range(len(pmf))) - u
        return eq

    rd.seed(whi_seed)
    us = [rd.random() for i in range(n)]
    xs = [fsolve(lambda x: get_eq(u, x), 0.1)[0] for u in us]
    return xs, us

In [58]:
sim_exp_hyper([0.8, 0.2], [0.8333, 5], 10)

([0.05401830386656942,
  0.0919086343974151,
  0.5497721085412586,
  0.11503180204529538,
  3.9111143322202384,
  0.03902185816658256,
  0.8327913219325541,
  0.4189656623227467,
  2.6993979711363267,
  0.17652609015514029],
 [0.052363598850944326,
  0.08718667752263232,
  0.4072417636703983,
  0.10770023493843905,
  0.9011988779516946,
  0.0381536661023224,
  0.5362020400339269,
  0.33219769850967984,
  0.8520866189293687,
  0.1596623967219699])

In [50]:
rd.random(10)

TypeError: random() takes no arguments (1 given)